In [1]:
# Bring in (new) PNC cohort

import pickle
from pathlib import Path
import numpy as np

newdir = '/home/anton/Documents/Tulane/Research/ImageNomer/data/anton/cohorts/BSNIP'
newdemo = pickle.load(open(f'{newdir}/demographics.pkl', 'rb'))

newfc = dict()

for sub in newdemo['Age_cal']:
    f = f'{newdir}/fc/{sub}_task-unk_fc.npy'
    if not Path(f).exists():
        continue
    p = np.load(f)
    newfc[f'{sub}'] = p

print(len(newfc))

1244


In [2]:
# Angle estimate

import torch
import torch.nn as nn

import numpy as np
import matplotlib.pyplot as plt

pi = 3.14
pi2 = 2*pi

def tonp(t):
    return t.detach().cpu().numpy()

class AngleBasis(nn.Module):
    def __init__(self, mixn):
        super(AngleBasis, self).__init__()
        self.mixn = mixn
        self.thetas = nn.Parameter((pi*torch.rand(self.mixn,264)+pi/2).float().cuda())
        self.jitter = nn.Parameter(torch.ones(self.mixn,264).float().cuda())
        
    def project(self):
        with torch.no_grad():
            self.jitter[self.jitter < 0] = 0
            self.jitter[self.jitter > 1] = 1
        
    def phases(self):
        t0 = self.thetas.unsqueeze(2)
        t1 = self.thetas.unsqueeze(1)
        return t0-t1
    
    def jit(self):
        j0 = self.jitter.unsqueeze(2)
        j1 = self.jitter.unsqueeze(1)
        return j0*j1
    
    def ps(self, jitter=True):
        t = self.phases()
        p = torch.cos(t)
        if jitter:
            j = self.jit()
            p = j*p
        return p
    
    def dump(self):
        return dict(mixn=self.mixn, thetas=tonp(self.thetas), jitter=tonp(self.jitter))
    
    def psum(self):
        return torch.mean(self.ps(), axis=0)
    
    def pvec(self):
        a,b = np.triu_indices(264,1)
        p = self.psum()
        return p[a,b]
    
print('Complete')

Complete


In [ ]:
nepochs = 10000
pperiod = 500

def fromnp(x):
    return torch.from_numpy(x).float().cuda()

def rmse(yhat, y):
    if isinstance(yhat, np.ndarray):
        f = np.mean
    else:
        f = torch.mean
    return f((y-yhat)**2)**0.5

bases = dict()
i = 0

for subtask in newfc:
    x = fromnp(newfc[subtask])

    min_loss = 10
    sav = None

    for _ in range(2):
        basis = AngleBasis(10)
        optim = torch.optim.Adam(basis.parameters(), lr=1e-2, weight_decay=0)

        for e in range(nepochs):
            optim.zero_grad()
            xhat = basis.pvec()
            loss = rmse(xhat, x)
            loss.backward()
            optim.step()
            basis.project()
            if e == nepochs-1 or e % pperiod == 1:
                print(f'{e} {float(loss)}')

        if loss < min_loss:
            print('Saving')
            min_loss = float(loss)
            sav = basis
    
    bases[subtask] = sav.dump()
    i += 1
    
    print(f'Finished {i} {subtask}')
        
print('Complete')

1 0.34190982580184937
501 0.06969284266233444
1001 0.06342737376689911
1501 0.06096436455845833
2001 0.0594768188893795
2501 0.05842061713337898
3001 0.05756678059697151
3501 0.05458803474903107
4001 0.05215785279870033
4501 0.05052150785923004
5001 0.049176208674907684
5501 0.04768920689821243
6001 0.046198680996894836
6501 0.04488332197070122
7001 0.043874043971300125
7501 0.04322023689746857
8001 0.04258928820490837
8501 0.041914839297533035
9001 0.04147715121507645
9501 0.04146282747387886
9999 0.04121853783726692
Saving
1 0.33675044775009155
501 0.06815320998430252
1001 0.06291673332452774
1501 0.05920438840985298
2001 0.05560551956295967
2501 0.053353387862443924
3001 0.05176683887839317
3501 0.050653405487537384
4001 0.048866935074329376
4501 0.047465257346630096
5001 0.0465758815407753
5501 0.0459284782409668
6001 0.045415252447128296
6501 0.04521498084068298
7001 0.04512928053736687
7501 0.04512931406497955
8001 0.045047562569379807
8501 0.04500674456357956
9001 0.045000419020

1001 0.06738414615392685
1501 0.06479686498641968
2001 0.06298120319843292
2501 0.060498595237731934
3001 0.058383602648973465
3501 0.05699628219008446
4001 0.05627964809536934
4501 0.05537725239992142
5001 0.05472510680556297
5501 0.05418197810649872
6001 0.05300142243504524
6501 0.05193616449832916
7001 0.051568981260061264
7501 0.05066826567053795
8001 0.0502106137573719
8501 0.050113629549741745
9001 0.050004635006189346
9501 0.04990961402654648
9999 0.04990427941083908
Finished 8 Baltimore.S0316VHT1
1 0.3539530634880066
501 0.07167087495326996
1001 0.06511859595775604
1501 0.06291361153125763
2001 0.060227978974580765
2501 0.05761873349547386
3001 0.05592315271496773
3501 0.054501574486494064
4001 0.053074534982442856
4501 0.052285850048065186
5001 0.05142001435160637
5501 0.05020470172166824
6001 0.04840224236249924
6501 0.048001863062381744
7001 0.04774939641356468
7501 0.04725055396556854
8001 0.046344611793756485
8501 0.04481439292430878
9001 0.04420536011457443
9501 0.0441530

1001 0.0699256882071495
1501 0.06771425157785416
2001 0.06642668694257736
2501 0.06492658704519272
3001 0.06385038048028946
3501 0.06256107240915298
4001 0.06134552136063576
4501 0.060149598866701126
5001 0.05880345031619072
5501 0.05755896121263504
6001 0.05707111209630966
6501 0.056354835629463196
7001 0.055730365216732025
7501 0.05518113076686859
8001 0.054868247359991074
8501 0.054475340992212296
9001 0.05355902016162872
9501 0.05290684849023819
9999 0.05178719386458397
Saving
1 0.37267571687698364
501 0.07609532028436661
1001 0.07033713161945343
1501 0.06772852689027786
2001 0.06564326584339142
2501 0.0637466087937355
3001 0.06218749284744263
3501 0.061730097979307175
4001 0.06094293296337128
4501 0.06030029430985451
5001 0.05967224761843681
5501 0.058318037539720535
6001 0.05733316019177437
6501 0.05574917793273926
7001 0.05448218807578087
7501 0.05392947793006897
8001 0.0534341037273407
8501 0.05310806632041931
9001 0.05270594358444214
9501 0.051917027682065964
9999 0.0516534596

1501 0.052632782608270645
2001 0.05054827407002449
2501 0.04920440539717674
3001 0.04862085357308388
3501 0.04779348522424698
4001 0.04621189832687378
4501 0.045507706701755524
5001 0.04519002139568329
5501 0.04454333335161209
6001 0.04341739043593407
6501 0.04278988763689995
7001 0.041411757469177246
7501 0.04111438989639282
8001 0.04046349972486496
8501 0.040149275213479996
9001 0.03971829637885094
9501 0.0395672507584095
9999 0.03903701528906822
Saving
Finished 23 Baltimore.S0926USC1
1 0.3233988881111145
501 0.06283871829509735
1001 0.058407507836818695
1501 0.05507614091038704
2001 0.053364451974630356
2501 0.05119766294956207
3001 0.05010083317756653
3501 0.050012655556201935
4001 0.049968644976615906
4501 0.049590855836868286
5001 0.048865508288145065
5501 0.04853576421737671
6001 0.04758540168404579
6501 0.04711122810840607
7001 0.046966783702373505
7501 0.045266035944223404
8001 0.045246873050928116
8501 0.04524700716137886
9001 0.04524683579802513
9501 0.045247454196214676
999

1001 0.05904152989387512
1501 0.05688409507274628
2001 0.05430198833346367
2501 0.05260387808084488
3001 0.05107301473617554
3501 0.05059203505516052
4001 0.05052417144179344
4501 0.05027509853243828
5001 0.049642279744148254
5501 0.04836253821849823
6001 0.0471942238509655
6501 0.04694197699427605
7001 0.04601376876235008
7501 0.04509484022855759
8001 0.043631888926029205
8501 0.043040864169597626
9001 0.0429578572511673
9501 0.04295242577791214
9999 0.04295242950320244
Saving
1 0.34501829743385315
501 0.061318639665842056
1001 0.056261468678712845
1501 0.05488848313689232
2001 0.0534098744392395
2501 0.05183852091431618
3001 0.04989100620150566
3501 0.04843776673078537
4001 0.04755525290966034
4501 0.047080956399440765
5001 0.04703106731176376
5501 0.0469573438167572
6001 0.046697475016117096
6501 0.04638921469449997
7001 0.046360451728105545
7501 0.046152882277965546
8001 0.04613475501537323
8501 0.04569092020392418
9001 0.043249357491731644
9501 0.042187292128801346
9999 0.04215855

...

7501 0.05107283219695091
8001 0.04873840510845184
8501 0.04750673845410347
9001 0.04704730212688446
9501 0.04596890136599541
9999 0.044299107044935226
Saving
1 0.41324281692504883
501 0.09044892340898514
1001 0.08132632821798325
1501 0.07357165217399597
2001 0.06967636942863464
2501 0.0648811012506485
3001 0.06218256056308746
3501 0.059971846640110016
4001 0.056549109518527985
4501 0.05433371290564537
5001 0.052888043224811554
5501 0.052008602768182755
6001 0.051713258028030396
6501 0.05157458782196045
7001 0.05115988478064537
7501 0.05115411803126335
8001 0.05115361511707306
8501 0.05115117132663727
9001 0.05113590881228447
9501 0.051113519817590714
9999 0.05111358314752579
Finished 640 Dallas.S3182OTN1
1 0.2942964732646942
501 0.05095050856471062
1001 0.0457380972802639
1501 0.04452797397971153
2001 0.04355219751596451
2501 0.042691078037023544
3001 0.04174201563000679
3501 0.04114280268549919
4001 0.040474843233823776
4501 0.03995778411626816
5001 0.03889734297990799
5501 0.03838368

8501 0.02992507629096508
9001 0.02993055246770382
9501 0.029925070703029633
9999 0.029925070703029633
Saving
Finished 647 Dallas.S3544RBE1
1 0.2811358571052551
501 0.0454612672328949
1001 0.04205910116434097
1501 0.04040082171559334
2001 0.039072781801223755
2501 0.03841507062315941
3001 0.037691593170166016
3501 0.036882828921079636
4001 0.03638410195708275
4501 0.03610493987798691
5001 0.035637933760881424
5501 0.03528910502791405
6001 0.03498682379722595
6501 0.03448745235800743
7001 0.034108251333236694
7501 0.03339860215783119
8001 0.03320009261369705
8501 0.03318720683455467
9001 0.0331743024289608
9501 0.033174075186252594
9999 0.03317401558160782
Saving
1 0.2865831255912781
501 0.0464799702167511
1001 0.043065235018730164
1501 0.041160114109516144
2001 0.039369646459817886
2501 0.03869418054819107
3001 0.038484085351228714
3501 0.03725811466574669
4001 0.03586947172880173
4501 0.03580059856176376
5001 0.035796381533145905
5501 0.03568890690803528
6001 0.035568926483392715
6501 

9001 0.03779620677232742
9501 0.03759727254509926
9999 0.03721833601593971
Saving
1 0.2915757894515991
501 0.055790066719055176
1001 0.05180690437555313
1501 0.04896965250372887
2001 0.0476590096950531
2501 0.045556265860795975
3001 0.04475611820816994
3501 0.044094499200582504
4001 0.04264941066503525
4501 0.040608182549476624
5001 0.040121808648109436
5501 0.03947939723730087
6001 0.03888159990310669
6501 0.038558680564165115
7001 0.03834638372063637
7501 0.038344915956258774
8001 0.03826434910297394
8501 0.03825914487242699
9001 0.03825856000185013
9501 0.03825831040740013
9999 0.03824995830655098
Finished 655 Dallas.S4087OKB1
1 0.3867351710796356
501 0.08127819001674652
1001 0.07284484058618546
1501 0.07093319296836853
2001 0.06816809624433517
2501 0.06324832141399384
3001 0.05982911214232445
3501 0.05856429785490036
4001 0.05743882432579994
4501 0.056416258215904236
5001 0.05507369711995125
5501 0.052293311804533005
6001 0.051358819007873535
6501 0.049692023545503616
7001 0.048536

501 0.04871930927038193
1001 0.04210146889090538
1501 0.039936669170856476
2001 0.039444033056497574
2501 0.03831564635038376
3001 0.03734607622027397
3501 0.03554521128535271
4001 0.03432905673980713
4501 0.033318113535642624
5001 0.0325683057308197
5501 0.03223966807126999
6001 0.0320683978497982
6501 0.03200020268559456
7001 0.03179269656538963
7501 0.031781744211912155
8001 0.03178166225552559
8501 0.03177656605839729
9001 0.031743209809064865
9501 0.03174221143126488
9999 0.0317424014210701
Saving
1 0.3014526665210724
501 0.049637604504823685
1001 0.04407356306910515
1501 0.0416206531226635
2001 0.04018189385533333
2501 0.03853984922170639
3001 0.03823971748352051
3501 0.03782731667160988
4001 0.0358923077583313
4501 0.03405749052762985
5001 0.0333692729473114
5501 0.03288775309920311
6001 0.03236350417137146
6501 0.03189149126410484
7001 0.03167594224214554
7501 0.03119821287691593
8001 0.030475281178951263
8501 0.030282042920589447
9001 0.030197586864233017
9501 0.03019089251756

1501 0.0653831884264946
2001 0.06353981047868729
2501 0.06132818013429642
3001 0.0592416487634182
3501 0.05721395090222359
4001 0.05581571161746979
4501 0.055306654423475266
5001 0.05418771132826805
5501 0.05317292734980583
6001 0.05248432606458664
6501 0.05025815963745117
7001 0.049135833978652954
7501 0.04821432754397392
8001 0.047069694846868515
8501 0.04593733698129654
9001 0.04476919770240784
9501 0.04402897506952286
9999 0.043298136442899704
Saving
Finished 670 Dallas.S4625LAL1
1 0.3882722854614258
501 0.07669462263584137
1001 0.0704057514667511
1501 0.06841007620096207
2001 0.06720113009214401
2501 0.0666189044713974
3001 0.0655035600066185
3501 0.06497509032487869
4001 0.0648564025759697
4501 0.06440748274326324
5001 0.06267551332712173
5501 0.060973696410655975
6001 0.05745542794466019
6501 0.05517035722732544
7001 0.05461277440190315
7501 0.053747180849313736
8001 0.05141939967870712
8501 0.049870166927576065
9001 0.049507275223731995
9501 0.04826422408223152
9999 0.046894479

In [12]:
import pickle

pickle.dump(bases, open('/home/anton/Documents/Tulane/Research/BSNIP/AngleBasisLong10.pkl', 'wb'))

print('Complete')

Complete
